In [7]:
import numpy as np
from numpy.random import rand
import math
from random import randint
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [252]:
class ConnectN:
    
    def __init__(self, grid_size, n):
        self.n = n
        self.grid_size = grid_size
        self.grid = np.zeros([grid_size,grid_size])
        self.finished = 0
        self.turn_num = 0
        
    def reset(self):
        self.__init__(self.grid_size, self.n)

    def check_win(self, col, row, player):
        for i in range(0, self.n):
            if sum(self.grid[col, row - i:row - i + self.n]) == self.n*player:
                self.finished = 1
                return 1
            if sum(self.grid[col - i: col - i + self.n, row]) == self.n*player:
                self.finished = 1
                return 1
            if col - i >= 0 and col - i + self.n - 1 < self.grid_size and row - i >= 0 and row - i + self.n - 1 < self.grid_size:
                if sum([self.grid[col - i + x, row - i + x] for x in range(0, self.n)]) == self.n*player:
                    self.finished = 1
                    return 1
            if col - i >= 0 and col - i + self.n - 1 < self.grid_size and row + i >= 0 and row + i - self.n + 1 < self.grid_size:
                if sum([self.grid[col - i + x, row + i - x] for x in range(0, self.n)]) == self.n*player:
                    self.finished = 1
                    return 1
        return 0

    def move(self, col, player):
        
        self.turn_num += 1
        
        if self.finished == 1:
            return 1, 50
        sum_col = np.sum([abs(x) for x in self.grid[col]])
        if sum_col == self.grid_size:
            return -1, -1
        self.grid[col, sum_col] = player
        if self.check_win(col, sum_col, player) == 1:
            return 1, 50
        return 0, 0
    
    def turn(self):
        """
        Returns which player's turn it is. First turn is player 1, second turn is player -1.
        """
        if self.turn_num%2 == 0:
            return 1
        else:
            return -1
        
    def next_possible_moves(self):
        """
        Returns array of possible columns for a next move
        """
        columns = []
        
        for i in xrange(0, self.grid_size):
            if (0 in self.grid[i]):
                columns.append(i)
                
        return columns
    
    def all_tokens_placed(self):
        """
        Returns location of all tokens (column, row) that have been placed
        """
        all_tokens = []
        
        for col in xrange(0, self.grid_size):
            for row in xrange(0, self.grid_size): 
                if self.grid[col][row] != 0:
                    all_tokens.append({"location": [col, row], "player": self.grid[col][row]})
                    
        return all_tokens
    
    def is_empty(self, col, row):
        return self.grid[col][row] == 0
    
    def print_grid(self):
        print(np.rot90(self.grid))

In [253]:
x = ConnectN(7, 5)
x.print_grid ()

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


In [254]:
x.next_possible_moves()

[0, 1, 2, 3, 4, 5, 6]

In [267]:
x.all_tokens_placed()

[{'location': [3, 0], 'player': 1.0},
 {'location': [3, 1], 'player': 1.0},
 {'location': [3, 2], 'player': 1.0},
 {'location': [3, 3], 'player': 1.0}]

In [265]:
x.move(3, 1)

(0, 0)

In [266]:
x.print_grid()

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]]


In [212]:
x.all_tokens_placed()

[{'location': [1, 0], 'player': 1.0}]

In [213]:
x.is_empty(1, 0)

False

In [214]:
x.grid[1][0]

1.0

### Minimax

In [393]:
import copy

In [674]:
class Minimax_Learner(object):
    """
    Simple implementation of AI algorithm Minimax with static evaluator 
    """
    
    def __init__(self, board, depth, n, player):
        self.board = board
        self.depth = depth
        self.num_states = board.grid_size
        self.player = player
        self.n = n
        
    def check_possible_wins(self, board):
    
        def check_vertical(token_dict):
            col = token_dict['location'][0]
            row = token_dict['location'][1]
            counter = 0
            
            for i in xrange(0, row):
                if board.is_empty(col, i) == False:
                    counter += 1
                    
            return counter
            
        def consecutive(condition):
            
            length,count = [], 0
            for i in range(len(condition)):

                if condition[i] == True:
                    count += 1
                elif condition[i] == False and count > 0:
                    length.append(count)
                    count = 0

                if i == len(condition) - 1 and count > 0:
                    length.append(count)

                return sum(length)
            
        def check_horizontal(token_dict):
            col = token_dict['location'][0]
            row = token_dict['location'][1]
            counter = 0
            
            rows = [board.grid[i,row] for i in xrange(0, self.num_states)]
            condition = [i != 0 for i in rows]
            counter += consecutive(condition)
                
            return counter 
        
        def check_diag_up(token_dict):
            col = token_dict['location'][0]
            row = token_dict['location'][1]
            
            # above the diagonal
            if col < row:
                start = row - col
                end = self.num_states
                
                diagonal = [board.grid[i+col-start][i] for i in xrange(start, end)]

            # below the diagonal
            elif row <= col:
                start = col - row
                end = self.num_states
                
                diagonal = [board.grid[i][i+row-start] for i in xrange(start, end)]
            
            condition = [i != 0 for i in diagonal]
            counter = consecutive(condition)
            
            return counter   
            
        def check_diag_down(token_dict):
            col = token_dict['location'][0]
            row = token_dict['location'][1]
            
            # above the diagonal
            if row > -col + self.num_states:
                start = row + col - self.num_states 
                end = self.num_states
                
                diagonal = [board.grid[start+i][end-i] for i in xrange(start, end+1)]

            # below the diagonal
            else:
                start = 0
                end = row + col
                
                diagonal = [board.grid[start+i][end-i] for i in xrange(start, end+1)]
                print diagonal
            
            condition = [i != 0 for i in diagonal]
            counter = consecutive(condition)
            
            return counter 
        
        current_tokens = board.all_tokens_placed()
        win_paths = 0
        
        for token in current_tokens:
            win_paths += check_vertical(token)
            win_paths += check_horizontal(token)
            win_paths += check_diag_up(token)
            win_paths += check_diag_down(token)
            
        return win_paths
        
    def create_tree(self, node, depth, player, move):
        
        if depth == 0:
            return None
        
        else:
            tree = {"value": 0, "children": [], "board": node, "player": player, "move": move}

            next_moves = node.next_possible_moves()

            for move in next_moves:
                board_copy = copy.deepcopy(node)
                board_copy.move(move, self.player)
                new_child = self.create_tree(board_copy, depth-1, 1-player, move)
                
                if new_child != None:
                    tree["children"].append(new_child)

            return tree

    def children(self, node):
        """ 
        returns children of a node
        """
        return node["children"]
   
    def leaf(self, node):
        """
        returns if current node is a leaf (i.e. no children)
        """
        return len(self.children(node)) == 0
        
    def max_node(self, node):
        """
        returns true if node is a max node and false if a node
        is a min node
        """
        return node["player"] == self.player
        
    def evaluate(self, node):
        """
        Static evaluator function to return a value between Loss and Win for intermediate game
        positions, larger if the position is better for the current player.
        If depth limit of the search is exceeded, is applied to remaining nodes as if
        they were leaves. 
        
        We calculate the rating by checking each token already placed, and 
        checking how many possible ways to connect N there are
        """
        node["value"] = self.check_possible_wins(node["board"])
        return node["value"]       

    def minimax(self, node, depth):
        """ 
        Recursive implementation of Minimax algorithm using pseudocode from: 
        https://www.cs.cornell.edu/courses/cs312/2002sp/lectures/rec21.htm
        
        Loss and Win values should be set to - and + infinity, respectively, but
        here use -/+ 1000
        """
        
        if self.leaf(node) or depth == 0:
            return self.evaluate(node)
        
        if self.max_node(node):
            # L = -infinity
            current_node_value = -1000
            for child in self.children(node):
                next_node_value = self.minimax(child, depth-1)
                if current_node_value < next_node_value:
                    current_node_value = next_node_value
            node["value"] = current_node_value
            return current_node_value
        
        if not self.max_node(node):
            # W = +infinity
            current_node_value = 1000
            for child in self.children(node):
                next_node_value = self.minimax(child, depth-1)
                if next_node_value < current_node_value:
                    current_node_value = next_node_value
            node["value"] = current_node_value
            return current_node_value

    def calc_next_move(self):
        
        current_tree = self.create_tree(self.board, self.depth, self.player, None)
        top_val = self.minimax(current_tree, self.depth)
        print "this is top_val", top_val
        
        #print current_tree- works
        for child in current_tree["children"]:
            #print child
            if child["value"] == top_val:
                #print "i'm here"
                return child["move"]
        

In [675]:
x = ConnectN(7, 4)
x.print_grid()

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


In [676]:
x.move(0, 1)
x.move(1, 1)
x.move(1, 1)
x.move(2, 1)
x.move(2, 1)

(0, 0)

In [677]:
x.print_grid()

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.  0.]]


In [678]:
test = Minimax_Learner(x, 3, 4, 1)  